In [ ]:
from langgraph.graph import StateGraph , START , END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [ ]:
load_dotenv()

model = ChatOpenAI()

In [ ]:
class JokeState(TypedDict):

  topic : str
  joke : str
  explanation : str


In [ ]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = model.invoke(prompt).content

    return {'joke': response}

In [ ]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = model.invoke(prompt).content

    return {'explanation': response}

In [ ]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke',generate_joke)
graph.add_node('generate_explanation',generate_explanation)

graph.add_edge(START , 'generate_joke')
graph.add_edge('generate_joke' , 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [ ]:
config1 = {"configurable" : {"thread_id" : "1"}}
workflow.invoke({"topic": "pizza" }, config=config1)

In [ ]:
workflow.get_state(config1)

In [ ]:
list[workflow.get_state_history(config1)]

### All intermediate state value through persistance

In [ ]:
config2 = {"configurable" : {"thread_id" : "2"}}
workflow.invoke({"topic": "pasta" }, config=config2)

In [ ]:
workflow.get_state(config2)

In [ ]:
list[workflow.get_state_history(config2)]